In [1]:
# MIT license: https://opensource.org/licenses/MIT
# See https://github.com/Isotel/mixedsim/blob/master/python/ngspice_read.py
# for a more complete library. Isotel's version is GPL licensed
from __future__ import division
import numpy as np
BSIZE_SP = 512 # Max size of a line of data; we don't want to read the
               # whole file to find a line, in case file does not have
               # expected structure.
MDATA_LIST = [b'title', b'date', b'plotname', b'flags', b'no. variables',
              b'no. points', b'dimensions', b'command', b'option']

def rawread(fname: str):
    """Read ngspice binary raw files. Return tuple of the data, and the
    plot metadata. The dtype of the data contains field names. This is
    not very robust yet, and only supports ngspice.
    >>> darr, mdata = rawread('test.py')
    >>> darr.dtype.names
    >>> plot(np.real(darr['frequency']), np.abs(darr['v(out)']))
    """
    # Example header of raw file
    # Title: rc band pass example circuit
    # Date: Sun Feb 21 11:29:14  2016
    # Plotname: AC Analysis
    # Flags: complex
    # No. Variables: 3
    # No. Points: 41
    # Variables:
    #         0       frequency       frequency       grid=3
    #         1       v(out)  voltage
    #         2       v(in)   voltage
    # Binary:
    fp = open(fname, 'rb')
    plot = {}
    count = 0
    arrs = []
    plots = []
    while (True):
        try:
            mdata = fp.readline(BSIZE_SP).split(b':', maxsplit=1)
        except:
            raise
        if len(mdata) == 2:
            if mdata[0].lower() in MDATA_LIST:
                plot[mdata[0].lower()] = mdata[1].strip()
            if mdata[0].lower() == b'variables':
                nvars = int(plot[b'no. variables'])
                npoints = int(plot[b'no. points'])
                plot['varnames'] = []
                plot['varunits'] = []
                for varn in range(nvars):
                    varspec = (fp.readline(BSIZE_SP).strip()
                               .decode('ascii').split())
                    assert(varn == int(varspec[0]))
                    plot['varnames'].append(varspec[1])
                    plot['varunits'].append(varspec[2])
            if mdata[0].lower() == b'binary':
                rowdtype = np.dtype({'names': plot['varnames'],
                                     'formats': [np.complex_ if b'complex'
                                                 in plot[b'flags']
                                                 else np.float_]*nvars})
                # We should have all the metadata by now
                arrs.append(np.fromfile(fp, dtype=rowdtype, count=npoints))
                plots.append(plot)
                fp.readline() # Read to the end of line
        else:
            break
    return (arrs, plots)

if __name__ == '__main__':
    arrs, plots = rawread('test.raw')
    print(arrs)

# Local Variables:
# mode: python
# End:

[array([(1.8, 0.3, 1.79999999, 1.79998882, 0.25846606, 0.25846606, 0.25846606, 0.25846606, 1.79998882, 1.79998882, 1.79998882, 0., -9.51843049e-11)],
      dtype=[('v(vdd)', '<f8'), ('v(vin)', '<f8'), ('v(vout)', '<f8'), ('v(net1)', '<f8'), ('v(net2)', '<f8'), ('v(m.xm2.msky130_fd_pr__nfet_01v8#dbody)', '<f8'), ('v(m.xm2.msky130_fd_pr__nfet_01v8#body)', '<f8'), ('v(m.xm2.msky130_fd_pr__nfet_01v8#sbody)', '<f8'), ('v(m.xm1.msky130_fd_pr__pfet_01v8#dbody)', '<f8'), ('v(m.xm1.msky130_fd_pr__pfet_01v8#body)', '<f8'), ('v(m.xm1.msky130_fd_pr__pfet_01v8#sbody)', '<f8'), ('i(vin)', '<f8'), ('i(vdd)', '<f8')])]
